# Plating of cells from 96-well plates
## Protocol Initialization
Ensure you have calibrated the pipette before doing this protocol

In [ ]:
from opentrons import simulate, execute
import json
import time

When running the protocol with a robot, replace "simulate" below with "execute"

In [ ]:
protocol = simulate.get_protocol_api('2.12')
protocol.home()

## Declaring Labware
- **Right mount**: A p300 pipette
- **Slot 10**: A 96-well PCR plate with cells
- **Slots 1 to 9**: 6-well plates up to the necessary number
- **Slot 11**: Tiprack for the pipette

Replace with appropriate labware codes in the cell below if needed
If you need to use custom labware, use our [Jupyter Notebook protocols tutorial](https://openplant.github.io/openplant_automation_protocols/Tutorials/Jupyter%20Notebooks/)

In [ ]:
tipracks_300 = [protocol.load_labware('opentrons_96_tiprack_300ul', '11')]

cells_plate = protocol.load_labware('biorad_96_wellplate_200ul_pcr', '10')
media_plates = [protocol.load_labware('corning_6_wellplate_16.8ml_flat', str(num + 1)) for num in range(9)]

p300 = protocol.load_instrument('p300_single_gen2', 'right', tip_racks=tipracks_300)

# Define the number of samples here
- Run the next cell and fill out the prompts
- You can use from 1 to 96 samples

In [ ]:
samples_no = int(input("Number of samples:"))
dispensing_volume = int(input("What volume to dispense in each plate well? (in uL, max 200)"))
print()
changes = 0

if samples_no - 1 in range(96):
    print(samples_no, 'samples will require', samples_no//6, 'plates')
    if samples_no > 54:
        print('with one plate change')
else:
    samples_no = 0
    print('Invalid number of samples')

## Labware Calibration

Change the x, y, and z variables in the "set_offset" functions and run each cell repeatedly until the positioning is correct for that piece of labware. You will have to use a "dummy" plate to calibrate the labware that is not on the deck yet (competent cells).

### Tipracks:

In [ ]:
protocol.comment('--- CALIBRATION ---')

In [ ]:
tipracks_300[0].set_offset(x=0.0,y=0.0,z=0.0)
p300.move_to(tipracks_300[0]['A1'].top())

### Plates/Racks:

no calibration for the 6-well plates

In [ ]:
p300.pick_up_tip()

In [ ]:
cells_plate.set_offset(x=0.0,y=0.0,z=0.0)
p300.move_to(cells_plate['A1'].top())

In [ ]:
p300.return_tip()
p300.reset_tipracks()

## Protocol Execution
run only the cell below repeatedly, up to the necessary number of plate changes

In [ ]:
protocol.comment('--- PLATING CELLS PASS N° '+ str(changes + 1) +' ---')
for sample in range(samples_no):
    protocol.comment('')
    cur_plate = sample//6
    print('Dispensing sample', sample+changes*54, 'on plate', cur_plate, end='\r')
    p300.pick_up_tip()
    p300.aspirate(dispensing_volume, cells_plate.wells()[sample + 54 * changes])
    p300.dispense(dispensing_volume, media_plates[cur_plate].wells()[sample % 6].top())
    protocol.delay(seconds=2)
    p300.blow_out()
    p300.blow_out()
    p300.drop_tip()
    if sample == samples_no - 1:
        print('\nDone.')
    elif sample == 53:
        samples_no -= 54
        changes += 1
        print('Change plates then run this cell again')
        break

In [ ]:
protocol.home()

In [ ]:
protocol.commands()